In [123]:
import sys, os, json, time, datetime
import rasterio, boto3, pystac, geojson

import pandas as pd

from shapely.geometry import shape, Polygon
from pystac import Catalog
from tqdm import tqdm
from satstac import Catalog, Collection, Item
from botocore import UNSIGNED
from botocore.config import Config
from GOSTRocks.misc import tPrint
from tqdm.notebook import tqdm, trange

sys.path.append("../src")
import stac_tools

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Define baseline variables
bucket = 'globalnightlight'
region = 'us-east-1'
s3client = boto3.client('s3', region_name=region, config=Config(signature_version=UNSIGNED))
existing_catalog = pd.read_csv("../src/catalog/VIIRS_Catalog_Final.csv", index_col=0)
existing_catalog.head()

,col_id,filename,item_id,href,bbox,geometry,date,datetime
0,VIIRS_DNB_rade9_201204,https://globalnightlight.s3.amazonaws.com/2012...,SVDNB_npp_d20120401_t0009017_e0014421_b02208_c...,https://globalnightlight.s3.amazonaws.com/2012...,"[-5.00208333335, 61.00208322135, 119.997917666...","POLYGON ((-5.002083 61.002083, -5.002083 75.00...",4/1/2012,2012-04-01 00:09:01+00:00
1,VIIRS_DNB_rade9_201204,https://globalnightlight.s3.amazonaws.com/2012...,SVDNB_npp_d20120401_t0014433_e0020237_b02208_c...,https://globalnightlight.s3.amazonaws.com/2012...,"[-2.002083335, 46.002061735, 69.99797426500001...","POLYGON ((-2.002083 46.002062, -2.002083 73.00...",4/1/2012,2012-04-01 00:14:43+00:00
2,VIIRS_DNB_rade9_201204,https://globalnightlight.s3.amazonaws.com/2012...,SVDNB_npp_d20120401_t0020249_e0026035_b02208_c...,https://globalnightlight.s3.amazonaws.com/2012...,"[2.997916665, 28.002062535000004, 46.997951865...","POLYGON ((2.997917 28.002063, 2.997917 54.0020...",4/1/2012,2012-04-01 00:20:24+00:00
3,VIIRS_DNB_rade9_201204,https://globalnightlight.s3.amazonaws.com/2012...,SVDNB_npp_d20120401_t0026047_e0031451_b02208_c...,https://globalnightlight.s3.amazonaws.com/2012...,"[-0.002083335, 9.002063335000003, 35.997945465...","POLYGON ((-0.002083 9.002063, -0.002083 34.002...",4/1/2012,2012-04-01 00:26:04+00:00
4,VIIRS_DNB_rade9_201204,https://globalnightlight.s3.amazonaws.com/2012...,SVDNB_npp_d20120401_t0031464_e0037268_b02208_c...,https://globalnightlight.s3.amazonaws.com/2012...,"[-4.002083335, -9.997935865, 28.99794306499999...","POLYGON ((-4.002083 -9.997935999999999, -4.002...",4/1/2012,2012-04-01 00:31:46+00:00


In [4]:
# Loop through the S3 bucket and get all the keys for files that are radiance tifs and json files
more_results = True
try:
    del(token)
except:
    pass
loops = 0
good_tif = []
good_json = []
while more_results:
    print(f"Completed loop: {loops}")
    if loops > 0:
        objects = s3client.list_objects_v2(Bucket=bucket, ContinuationToken=token)
    else:
        objects = s3client.list_objects_v2(Bucket=bucket)
    more_results = objects['IsTruncated']
    if more_results:
        token = objects['NextContinuationToken']
    loops += 1
    for res in objects['Contents']:
        if res['Key'].endswith('rade9.co.tif'):
            good_tif.append(res)
        if res['Key'].endswith('.json'):
            good_json.append(res)

Completed loop: 0
Completed loop: 1
Completed loop: 2
Completed loop: 3
Completed loop: 4
Completed loop: 5
Completed loop: 6
Completed loop: 7
Completed loop: 8
Completed loop: 9
Completed loop: 10
Completed loop: 11
Completed loop: 12
Completed loop: 13
Completed loop: 14
Completed loop: 15
Completed loop: 16
Completed loop: 17
Completed loop: 18
Completed loop: 19
Completed loop: 20
Completed loop: 21
Completed loop: 22
Completed loop: 23
Completed loop: 24
Completed loop: 25
Completed loop: 26
Completed loop: 27
Completed loop: 28
Completed loop: 29
Completed loop: 30
Completed loop: 31
Completed loop: 32
Completed loop: 33
Completed loop: 34
Completed loop: 35
Completed loop: 36
Completed loop: 37
Completed loop: 38
Completed loop: 39
Completed loop: 40
Completed loop: 41
Completed loop: 42
Completed loop: 43
Completed loop: 44
Completed loop: 45
Completed loop: 46
Completed loop: 47
Completed loop: 48
Completed loop: 49
Completed loop: 50
Completed loop: 51
Completed loop: 52
Com

Completed loop: 417
Completed loop: 418
Completed loop: 419
Completed loop: 420
Completed loop: 421
Completed loop: 422
Completed loop: 423
Completed loop: 424
Completed loop: 425
Completed loop: 426
Completed loop: 427
Completed loop: 428
Completed loop: 429
Completed loop: 430
Completed loop: 431
Completed loop: 432
Completed loop: 433
Completed loop: 434
Completed loop: 435
Completed loop: 436
Completed loop: 437
Completed loop: 438
Completed loop: 439
Completed loop: 440
Completed loop: 441
Completed loop: 442
Completed loop: 443
Completed loop: 444
Completed loop: 445
Completed loop: 446
Completed loop: 447
Completed loop: 448
Completed loop: 449
Completed loop: 450
Completed loop: 451
Completed loop: 452
Completed loop: 453
Completed loop: 454
Completed loop: 455
Completed loop: 456
Completed loop: 457
Completed loop: 458
Completed loop: 459
Completed loop: 460
Completed loop: 461
Completed loop: 462
Completed loop: 463
Completed loop: 464
Completed loop: 465
Completed loop: 466


Completed loop: 827
Completed loop: 828
Completed loop: 829
Completed loop: 830
Completed loop: 831
Completed loop: 832
Completed loop: 833
Completed loop: 834
Completed loop: 835
Completed loop: 836
Completed loop: 837
Completed loop: 838
Completed loop: 839
Completed loop: 840
Completed loop: 841
Completed loop: 842
Completed loop: 843
Completed loop: 844
Completed loop: 845
Completed loop: 846
Completed loop: 847
Completed loop: 848
Completed loop: 849
Completed loop: 850
Completed loop: 851
Completed loop: 852
Completed loop: 853
Completed loop: 854
Completed loop: 855
Completed loop: 856
Completed loop: 857
Completed loop: 858
Completed loop: 859
Completed loop: 860
Completed loop: 861
Completed loop: 862
Completed loop: 863
Completed loop: 864
Completed loop: 865
Completed loop: 866
Completed loop: 867
Completed loop: 868
Completed loop: 869
Completed loop: 870
Completed loop: 871
Completed loop: 872
Completed loop: 873
Completed loop: 874
Completed loop: 875
Completed loop: 876


Completed loop: 1226
Completed loop: 1227
Completed loop: 1228
Completed loop: 1229
Completed loop: 1230
Completed loop: 1231
Completed loop: 1232
Completed loop: 1233
Completed loop: 1234
Completed loop: 1235
Completed loop: 1236
Completed loop: 1237
Completed loop: 1238
Completed loop: 1239
Completed loop: 1240
Completed loop: 1241
Completed loop: 1242
Completed loop: 1243
Completed loop: 1244
Completed loop: 1245
Completed loop: 1246
Completed loop: 1247
Completed loop: 1248
Completed loop: 1249
Completed loop: 1250
Completed loop: 1251
Completed loop: 1252
Completed loop: 1253
Completed loop: 1254
Completed loop: 1255
Completed loop: 1256
Completed loop: 1257
Completed loop: 1258
Completed loop: 1259
Completed loop: 1260
Completed loop: 1261
Completed loop: 1262
Completed loop: 1263
Completed loop: 1264
Completed loop: 1265
Completed loop: 1266
Completed loop: 1267
Completed loop: 1268
Completed loop: 1269
Completed loop: 1270
Completed loop: 1271
Completed loop: 1272
Completed loo

Completed loop: 1617
Completed loop: 1618
Completed loop: 1619
Completed loop: 1620
Completed loop: 1621
Completed loop: 1622
Completed loop: 1623
Completed loop: 1624
Completed loop: 1625
Completed loop: 1626
Completed loop: 1627
Completed loop: 1628
Completed loop: 1629
Completed loop: 1630
Completed loop: 1631
Completed loop: 1632
Completed loop: 1633
Completed loop: 1634
Completed loop: 1635
Completed loop: 1636
Completed loop: 1637
Completed loop: 1638
Completed loop: 1639
Completed loop: 1640
Completed loop: 1641
Completed loop: 1642
Completed loop: 1643
Completed loop: 1644
Completed loop: 1645
Completed loop: 1646
Completed loop: 1647
Completed loop: 1648
Completed loop: 1649
Completed loop: 1650
Completed loop: 1651
Completed loop: 1652
Completed loop: 1653
Completed loop: 1654
Completed loop: 1655
Completed loop: 1656
Completed loop: 1657
Completed loop: 1658
Completed loop: 1659
Completed loop: 1660
Completed loop: 1661
Completed loop: 1662
Completed loop: 1663
Completed loo

Completed loop: 2009
Completed loop: 2010
Completed loop: 2011
Completed loop: 2012
Completed loop: 2013
Completed loop: 2014
Completed loop: 2015
Completed loop: 2016
Completed loop: 2017
Completed loop: 2018
Completed loop: 2019
Completed loop: 2020
Completed loop: 2021
Completed loop: 2022
Completed loop: 2023
Completed loop: 2024
Completed loop: 2025
Completed loop: 2026
Completed loop: 2027
Completed loop: 2028
Completed loop: 2029
Completed loop: 2030
Completed loop: 2031
Completed loop: 2032
Completed loop: 2033
Completed loop: 2034
Completed loop: 2035
Completed loop: 2036
Completed loop: 2037
Completed loop: 2038
Completed loop: 2039
Completed loop: 2040
Completed loop: 2041
Completed loop: 2042
Completed loop: 2043
Completed loop: 2044
Completed loop: 2045
Completed loop: 2046
Completed loop: 2047
Completed loop: 2048
Completed loop: 2049
Completed loop: 2050
Completed loop: 2051
Completed loop: 2052
Completed loop: 2053
Completed loop: 2054
Completed loop: 2055
Completed loo

Completed loop: 2401
Completed loop: 2402
Completed loop: 2403
Completed loop: 2404
Completed loop: 2405
Completed loop: 2406
Completed loop: 2407
Completed loop: 2408
Completed loop: 2409
Completed loop: 2410
Completed loop: 2411
Completed loop: 2412
Completed loop: 2413
Completed loop: 2414
Completed loop: 2415
Completed loop: 2416
Completed loop: 2417
Completed loop: 2418
Completed loop: 2419
Completed loop: 2420
Completed loop: 2421
Completed loop: 2422
Completed loop: 2423
Completed loop: 2424
Completed loop: 2425
Completed loop: 2426
Completed loop: 2427
Completed loop: 2428
Completed loop: 2429
Completed loop: 2430
Completed loop: 2431
Completed loop: 2432
Completed loop: 2433
Completed loop: 2434
Completed loop: 2435
Completed loop: 2436
Completed loop: 2437
Completed loop: 2438
Completed loop: 2439
Completed loop: 2440
Completed loop: 2441
Completed loop: 2442
Completed loop: 2443
Completed loop: 2444
Completed loop: 2445
Completed loop: 2446
Completed loop: 2447
Completed loo

Completed loop: 2792
Completed loop: 2793
Completed loop: 2794
Completed loop: 2795
Completed loop: 2796
Completed loop: 2797
Completed loop: 2798
Completed loop: 2799
Completed loop: 2800
Completed loop: 2801
Completed loop: 2802
Completed loop: 2803
Completed loop: 2804
Completed loop: 2805
Completed loop: 2806
Completed loop: 2807
Completed loop: 2808
Completed loop: 2809
Completed loop: 2810
Completed loop: 2811
Completed loop: 2812
Completed loop: 2813
Completed loop: 2814
Completed loop: 2815
Completed loop: 2816
Completed loop: 2817
Completed loop: 2818
Completed loop: 2819
Completed loop: 2820
Completed loop: 2821
Completed loop: 2822
Completed loop: 2823
Completed loop: 2824
Completed loop: 2825
Completed loop: 2826
Completed loop: 2827
Completed loop: 2828
Completed loop: 2829
Completed loop: 2830
Completed loop: 2831
Completed loop: 2832
Completed loop: 2833
Completed loop: 2834
Completed loop: 2835
Completed loop: 2836
Completed loop: 2837
Completed loop: 2838
Completed loo

Completed loop: 3184
Completed loop: 3185
Completed loop: 3186
Completed loop: 3187
Completed loop: 3188
Completed loop: 3189
Completed loop: 3190
Completed loop: 3191
Completed loop: 3192
Completed loop: 3193
Completed loop: 3194
Completed loop: 3195
Completed loop: 3196
Completed loop: 3197
Completed loop: 3198
Completed loop: 3199
Completed loop: 3200
Completed loop: 3201
Completed loop: 3202
Completed loop: 3203
Completed loop: 3204
Completed loop: 3205
Completed loop: 3206
Completed loop: 3207
Completed loop: 3208
Completed loop: 3209
Completed loop: 3210
Completed loop: 3211
Completed loop: 3212
Completed loop: 3213
Completed loop: 3214
Completed loop: 3215
Completed loop: 3216
Completed loop: 3217
Completed loop: 3218
Completed loop: 3219
Completed loop: 3220
Completed loop: 3221
Completed loop: 3222
Completed loop: 3223
Completed loop: 3224
Completed loop: 3225
Completed loop: 3226
Completed loop: 3227
Completed loop: 3228
Completed loop: 3229
Completed loop: 3230
Completed loo

Completed loop: 3575
Completed loop: 3576
Completed loop: 3577
Completed loop: 3578
Completed loop: 3579
Completed loop: 3580
Completed loop: 3581
Completed loop: 3582
Completed loop: 3583
Completed loop: 3584
Completed loop: 3585
Completed loop: 3586
Completed loop: 3587
Completed loop: 3588
Completed loop: 3589
Completed loop: 3590
Completed loop: 3591
Completed loop: 3592
Completed loop: 3593
Completed loop: 3594
Completed loop: 3595
Completed loop: 3596
Completed loop: 3597
Completed loop: 3598
Completed loop: 3599
Completed loop: 3600
Completed loop: 3601
Completed loop: 3602
Completed loop: 3603
Completed loop: 3604
Completed loop: 3605
Completed loop: 3606
Completed loop: 3607
Completed loop: 3608
Completed loop: 3609
Completed loop: 3610
Completed loop: 3611
Completed loop: 3612
Completed loop: 3613
Completed loop: 3614
Completed loop: 3615
Completed loop: 3616
Completed loop: 3617
Completed loop: 3618
Completed loop: 3619
Completed loop: 3620
Completed loop: 3621
Completed loo

In [5]:
good_json_keys = [os.path.basename(x['Key'])[:-5] for x in good_json]
good_tif_keys = [os.path.basename(x['Key'])[:-4] for x in good_tif]
missing_json = list(set(good_tif_keys) - set(good_json_keys))
print(f'Nightly files missing JSON: {len(missing_json)}')

Nightly files missing JSON: 32653


In [12]:
good_csv_keys = existing_catalog['item_id'].values
missing_csv = list(set(good_tif_keys) - set(good_csv_keys))
print(f'Nightly files missing in CSV: {len(missing_csv)}')

Nightly files missing in CSV: 23327


In [ ]:
# Update LEN STAC csv file
all_items = {}
catalog_items = []
cnt = 0
for jFile in missing_json:
    if cnt % 50 == 0:
        tPrint(f"{cnt} of {len(missing_json)}")
    jRes = stac_tools.v_file(jFile)
    catalog_items.append(jRes.cEntry)    
    cnt += 1

In [ ]:
column_names = ['col_id','filename','item_id','href','bbox','geometry','date','datetime']
new_items = pd.DataFrame(catalog_items, columns=column_names)
tempD = existing_catalog.copy()
tempD = tempD.append(new_items)
tempD.to_csv("../src/catalog/VIIRS_Catalog_Final_updated.csv")

# Update STAC JSON

In [120]:
out_folder = "/home/wb411133/temp/VIIRS_JSON/"

In [62]:
# Get list of input STAC_Folders in S3 bucket
stac_folders = [os.path.dirname(x['Key']) for x in good_tif]
stac_folders = list(set(stac_folders))

In [131]:
for cur_month in stac_folders:
    # Create paretn catalog for current month
    catalog = pystac.Catalog(id=catalog_id, description=f"VIIRS nighttime lights for {cur_month}")
    catalog.set_self_href(os.path.join(out_folder, f'{cur_month}.json'))
    # Find all tif files in the current month
    #all_images = stac_tools.search_aws(s3client, bucket, endswith = ".tif", prefix = f'{cur_month}/')
    all_tifs = [os.path.basename(x['Key']) for x in all_images]
    #Get complete list of overpass ids from file names
    all_ids = list(set(["_".join(x.split("_")[2:6]) for x in all_tifs]))
    all_ids = [x for x in all_ids if x[0] == 'd']
    #For each overpass, create catalog netry
    for c_id in tqdm(all_ids[:10]): #limited to firstt t10 items for testing
        sel_images = [x for x in all_tifs if c_id in x]
        cur_stac = stac_tools.v_file(sel_images, cur_month)
        catalog.add_item(cur_stac.pystac_item)
    
    catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)
    
    break

  0%|          | 0/10 [00:00<?, ?it/s]

GDNBO_npp_d20150626_t1141459_e1147263_b18970_c20150626174727370571_noaa_ops.li.co.tif
GDTCN_npp_d20150626_t1141459_e1147263_b18970_c20150626174727370571_noaa_ops.samples.co.tif
SVDNB_npp_d20150626_t1141459_e1147263_b18970_c20150626174727370798_noaa_ops.rade9.co.tif
SVM15_npp_d20150626_t1141459_e1147263_b18970_c20150626174727474658_noaa_ops.rad.co.tif
npp_d20150626_t1141459_e1147263_b18970.vflag.co.tif
GDNBO_npp_d20150617_t0416079_e0421483_b18838_c20150617102148113098_noaa_ops.li.co.tif
GDTCN_npp_d20150617_t0416079_e0421483_b18838_c20150617102148113098_noaa_ops.samples.co.tif
SVDNB_npp_d20150617_t0416079_e0421483_b18838_c20150617102148113329_noaa_ops.rade9.co.tif
SVM15_npp_d20150617_t0416079_e0421483_b18838_c20150617102148294241_noaa_ops.rad.co.tif
npp_d20150617_t0416079_e0421483_b18838.vflag.co.tif
GDNBO_npp_d20150625_t0840066_e0845470_b18954_c20150625144548004178_noaa_ops.li.co.tif
GDTCN_npp_d20150625_t0840066_e0845470_b18954_c20150625144548004178_noaa_ops.samples.co.tif
SVDNB_npp_d20

# DEBUGGING

In [ ]:
import pystac as stac
import rasterio
import os
from datetime import datetime
from shapely.geometry import Polygon, mapping

# function to get bounding box from raster
def get_bbox_and_footprint(raster_uri):
    with rasterio.open(raster_uri) as ds:
        bounds = ds.bounds
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        footprint = Polygon([
            [bounds.left, bounds.bottom],
            [bounds.left, bounds.top],
            [bounds.right, bounds.top],
            [bounds.right, bounds.bottom]
        ])
        return (bbox, mapping(footprint))

# set year, month, and directory (add 'npp_' starting in 2018 to differentiate)
year = '2018'
month = '01'
satdir = 'npp_' + year + month

# create catalog description and id for radiance files
stac_descrip = 'Visible radiance from VIIRS DNB for ' + year + '/' + month + '.'
catalog_id = 'VIIRS_DNB_rade9_' + satdir

# make catalog
catalog = stac.Catalog(id=catalog_id, description=stac_descrip)

# directory and full file path of catalog file to write locally
stac_location = '/path/to/VIIRS/STAC/catalogs/' + sat_dir + '/'
catalog_path = stac_location + sat_dir + '_catalog.json'

catalog.set_self_href(catalog_path)

# path of list of files to add as items
item_list = '/path/to/item/list/' + sat_dir + '_items.txt'

# open the list of files
with open(item_list, 'r') as file:
    item_paths = file.readlines()

# generate STAC json for each item
for tif in item_paths:
  # get item path
  img_path = tif.replace('\n', '')
  # get image name
  img_name = os.path.basename(img_path)
  
  # get bounding box
  bbox, footprint = get_bbox_and_footprint(img_path)
  
  # get date time from filename
  img_datetime_str = img_name[11:15] + '-' + img_name[15:17] + '-' + img_name[17:19] + ' ' + img_name[21:23] + ':' + img_name[23:25] + ':' + img_name[25:27]
  img_datetime = datetime.fromisoformat(img_datetime_str)
  
  # create item
  stac_name = img_name[0:-4]
  # make STAC item (add additional properties here)
  item = stac.Item(id=stac_name,
                   geometry=footprint,
                   bbox=bbox,
                   datetime=img_datetime,
                   properties={})
  
  # add item to catalog
  catalog.add_item(item)
  
  # add asset to catalog using full public AWS path
  img_URL = 'https://globalnightlight.s3.amazonaws.com/' + sat_dir + '/' + img_name
  # add asset
  item.add_asset(key='image', asset=stac.Asset(href=img_URL, media_type=stac.MediaType.COG))
  
  # set item file path for individual item json to write locally
  item_location = stac_location + stac_name + '.json'
  item.set_self_href(item_location)

# save catalog and write jsons
catalog.save(catalog_type=stac.CatalogType.SELF_CONTAINED)
